In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\shawn\\OneDrive\\Desktop\\Krish Naik MLOPS\\endtoend1\\datascienceproj\\research'

In [3]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\shawn\\OneDrive\\Desktop\\Krish Naik MLOPS\\endtoend1\\datascienceproj'

In [8]:
from dataclasses import dataclass
from pathlib import Path 
@dataclass
class DataTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float 
    l1_ratio: float
    target_column: str

In [7]:
from src.datascience.constants import * 
from src.datascience.utils.common import read_yaml,create_directories

In [13]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) ->DataTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([self.config.artifacts_root])


        model_trainer_config=DataTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_config


    
    

In [14]:
import pandas as pd 
from  src.datascience import logger 
from sklearn.linear_model import ElasticNet
import joblib

In [15]:
class ModelTrainer:
    def __init__(self,config: DataTrainerConfig):
        config=self.config 

    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)

        train_x=train_data.drop([self.config.target_column])
        test_x=test_data.drop([self.config.target_column])
        train_y=train_data[[self.config.target_column]]
        test_y=test_data[[self.config.target_column]]

        lr=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))

    